# Import packages and set envir

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
%matplotlib inline
import plotly.offline as py 
import plotly.graph_objs as go 

In [15]:
df_classes = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_classes.csv")
df_edges = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
df_features = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_features.csv", header=None)

In [16]:
df_features.columns = ['txId', 'time_step'] + [f'trans_feat_{i}' for i in range(93)] + [f'agg_feat_{i}' for i in range(72)]

In [17]:
df_features = pd.merge(df_features,df_classes,left_on="txId",right_on="txId",how='left')
df_features['class'] = df_features['class'].apply(lambda x: '0' if x == "unknown" else x)

# Graph

In [30]:
def plot_graph(time, category):
    bad_ids = df_features[(df_features['time_step'] == time) & ((df_features['class'] == category))]['txId']
    short_edges = df_edges[df_edges['txId1'].isin(bad_ids)]
    graph = nx.from_pandas_edgelist(short_edges, source = 'txId1', target = 'txId2', 
                                    create_using = nx.DiGraph())
    pos = nx.spring_layout(graph)

    edge_x = []
    edge_y = []
    for edge in graph.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='blue'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_text=[]
    for node in graph.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Transaction Type',
                xanchor='left',
                titleside='right',
                tickmode='array',
                tickvals=[0,1,2],
                ticktext=['Unknown','Illicit','Licit']
            ),
            line_width=2))
    node_trace.text=node_text
    node_trace.marker.color = pd.to_numeric(df_features[df_features['txId'].isin(list(graph.nodes()))]['class'])

    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title="Illicit Transactions" + str(time),
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        showarrow=True,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [31]:
for i in range(1,41):
    plot_graph(i, '1')